In [1]:
import pandas as pd

In [4]:
datapath="./Data/Cleansed/cleant_data.csv"
reviews=pd.read_csv(datapath)

In [5]:
reviews.head()

,Unnamed: 0,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,DateTime,Popularity,Year,binary_score,CleantText,Text_SWR
0,150528,150529,0006641040,A25ACLV5KPB4W,"Matt Hetling ""Matt""",0,1,4,1108425600,"Nice cadence, catchy rhymes",In June<br />I saw a charming group<br />of ro...,2005-02-15 01:00:00,-1,2005,1,in june i saw a charming group of roses all be...,june saw charming group roses begin droop pepp...
1,150506,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,4,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,2007-10-04 02:00:00,0,2007,1,this is a fun way for children to learn their ...,fun way children learn months year learn poems...
2,150505,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,4,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",2007-11-11 01:00:00,0,2007,1,i grew up reading these sendak books and watch...,grew reading sendak books watching really rosi...
3,150504,150505,0006641040,A2PTSM496CF40Z,"Jason A. Teeple ""Nobody made a greater mistak...",1,1,4,1210809600,A classic,Get the movie or sound track and sing along wi...,2008-05-15 02:00:00,0,2008,1,get the movie or sound track and sing along wi...,get movie sound track sing along carol king gr...
4,150503,150504,0006641040,AQEYF1AXARWJZ,"Les Sinclair ""book maven""",1,1,4,1212278400,Chicken Soup with Rice,A very entertaining rhyming story--cleaver and...,2008-06-01 02:00:00,0,2008,1,a very entertaining rhyming storycleaver and c...,entertaining rhyming storycleaver catchythe il...


In [13]:
reviews_gpt = reviews.drop(['Unnamed: 0', 'Id', 'UserId', 'ProfileName', 'Time', 'Summary', 'DateTime', 'Popularity', 'binary_score', 'Text','Text_SWR'], axis=1)

In [14]:
reviews_gpt.head()

,ProductId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Year,CleantText
0,0006641040,0,1,4,2005,in june i saw a charming group of roses all be...
1,0006641040,1,1,4,2007,this is a fun way for children to learn their ...
2,0006641040,1,1,4,2007,i grew up reading these sendak books and watch...
3,0006641040,1,1,4,2008,get the movie or sound track and sing along wi...
4,0006641040,1,1,4,2008,a very entertaining rhyming storycleaver and c...


In [16]:
import pandas as pd

Product with the highest mean score: 141278509X
Highest mean score: 5.0


# Summary - 25 most reviewed products with mean score 1 

In [268]:
import pandas as pd

# Group by 'ProductId' and count occurrences, then sort by occurrence
product_occurrences = reviews['ProductId'].value_counts().sort_values(ascending=False)

# Sort the DataFrame based on product occurrences
sorted_df = reviews[reviews['ProductId'].isin(product_occurrences.index)].sort_values(by='ProductId', key=lambda x: product_occurrences[x], ascending=False)

#Calculate mean score for each product after sorting
mean_scores = sorted_df.groupby('ProductId', sort=False)['Score'].mean()

#Filter products with mean score equal to 1 and select the first 5 products
top_low_scored_products = mean_scores[mean_scores == 1].head(25)

client = AzureOpenAI(
    api_key="fec8056ea62842b2b3e1f787171f4f33",  
    api_version="2023-12-01-preview",
    azure_endpoint = "https://azureaistudioimagegeneration.openai.azure.com/"
    )
    
concatenated_response = ""
# Step 4: Print the ID, score, number of occurrences
for product_id, low_score in top_low_scored_products.items():
    occurrences = product_occurrences.get(product_id, 0)
    top_50_reviews = sorted_df[sorted_df['ProductId'] == product_id].nlargest(50, 'HelpfulnessNumerator')['CleantText'].tolist()
    concatenated_reviews = ' '.join(top_50_reviews)
    print(f"Product ID: {product_id}, Score: {low_score}, Occurrences: {occurrences}")

    response = client.chat.completions.create(
        model="capstone", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are going to answer only from the following reviews data" + concatenated_reviews},
            {"role": "user", "content": "Guess and name the product. Just rate from 1 to 5 and no text for the following aspects, Quality, Price, Brand, Packaging, Shipping, Value, Satisfaction, Health, Taste, Flavour. Then summarize what is negatively shortly below"},
        ]
    )
    
    #print(response.choices[0].message.content)
    print()
    concatenated_response += response.choices[0].message.content
    
response = client.chat.completions.create(
        model="capstone", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are going to answer only from the following reviews data" + concatenated_response},
            {"role": "user", "content": "Give a general overview on the type of products, why rated negatively and what can be improved. Give average rating of Quality, Price, Brand, Packaging, Shipping, Value, Satisfaction, Health, Taste, Flavour as a unordered list. Just list the product names at the last and no other content"},
        ]
    )

print(response.choices[0].message.content)

Product ID: B004V6A9SC, Score: 1.0, Occurrences: 12

Product ID: B004OV3VGQ, Score: 1.0, Occurrences: 8

Product ID: B005QD0V8M, Score: 1.0, Occurrences: 7

Product ID: B004US0VMU, Score: 1.0, Occurrences: 6

Product ID: B004BNNO0U, Score: 1.0, Occurrences: 5

Product ID: B004BNJZZS, Score: 1.0, Occurrences: 5

Product ID: B001HTRDVM, Score: 1.0, Occurrences: 5

Product ID: B005EKWOZK, Score: 1.0, Occurrences: 5

Product ID: B0026A30PY, Score: 1.0, Occurrences: 5

Product ID: B00212EDVW, Score: 1.0, Occurrences: 5

Product ID: B000HQP2N2, Score: 1.0, Occurrences: 4

Product ID: B007IWEFW2, Score: 1.0, Occurrences: 4

Product ID: B000I1QO5Q, Score: 1.0, Occurrences: 4

Product ID: B000SAPX7W, Score: 1.0, Occurrences: 4

Product ID: B0006344NK, Score: 1.0, Occurrences: 4

Product ID: B001H32FP2, Score: 1.0, Occurrences: 4

Product ID: B002U56LG0, Score: 1.0, Occurrences: 4

Product ID: B000Q3ZPSI, Score: 1.0, Occurrences: 4

Product ID: B001OKDYFI, Score: 1.0, Occurrences: 4

Product ID:

# Summary - 25 most reviewed products with mean greater than 4 

In [269]:
import pandas as pd

# Group by 'ProductId' and count occurrences, then sort by occurrence
product_occurrences = reviews['ProductId'].value_counts().sort_values(ascending=False)

# Sort the DataFrame based on product occurrences
sorted_df = reviews[reviews['ProductId'].isin(product_occurrences.index)].sort_values(by='ProductId', key=lambda x: product_occurrences[x], ascending=False)

#Calculate mean score for each product after sorting
mean_scores = sorted_df.groupby('ProductId', sort=False)['Score'].mean()

#Filter products with mean score equal to 1 and select the first 5 products
top_low_scored_products = mean_scores[mean_scores > 4].head(25)

client = AzureOpenAI(
    api_key="fec8056ea62842b2b3e1f787171f4f33",  
    api_version="2023-12-01-preview",
    azure_endpoint = "https://azureaistudioimagegeneration.openai.azure.com/"
    )
    
concatenated_response = ""
# Step 4: Print the ID, score, number of occurrences
for product_id, low_score in top_low_scored_products.items():
    occurrences = product_occurrences.get(product_id, 0)
    top_50_reviews = sorted_df[sorted_df['ProductId'] == product_id].nlargest(50, 'HelpfulnessNumerator')['CleantText'].tolist()
    concatenated_reviews = ' '.join(top_50_reviews)
    print(f"Product ID: {product_id}, Score: {low_score}, Occurrences: {occurrences}")

    response = client.chat.completions.create(
        model="capstone", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are going to answer only from the following reviews data" + concatenated_reviews},
            {"role": "user", "content": "Guess and name the product. Just rate from 1 to 5 and no text for the following aspects, Quality, Price, Brand, Packaging, Shipping, Value, Satisfaction, Health, Taste, Flavour. Then summarize what is positive shortly below"},
        ]
    )
    
    #print(response.choices[0].message.content)
    print()
    concatenated_response += response.choices[0].message.content
    
response = client.chat.completions.create(
        model="capstone", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are going to answer only from the following reviews data" + concatenated_response},
            {"role": "user", "content": "Give a general overview on the type of products, why rated positively and what can be improved. Give average rating of Quality, Price, Brand, Packaging, Shipping, Value, Satisfaction, Health, Taste, Flavour as a unordered list. Just list the product names at the last and no other content"},
        ]
    )

print(response.choices[0].message.content)

Product ID: B007JFMH8M, Score: 4.582236842105263, Occurrences: 912

Product ID: B0026RQTGE, Score: 4.593650793650793, Occurrences: 630

Product ID: B003B3OOPA, Score: 4.739967897271268, Occurrences: 623

Product ID: B001EO5Q64, Score: 4.746902654867257, Occurrences: 565

Product ID: B000VK8AVK, Score: 4.307829181494662, Occurrences: 562

Product ID: B000UBD88A, Score: 4.343173431734317, Occurrences: 542

Product ID: B000NMJWZO, Score: 4.88909426987061, Occurrences: 541

Product ID: B005ZBZLT4, Score: 4.364541832669323, Occurrences: 502

Product ID: B000PDWBKO, Score: 4.3539094650205765, Occurrences: 486

Product ID: B0029XLH4Y, Score: 4.256578947368421, Occurrences: 456

Product ID: B0007A0AQM, Score: 4.542654028436019, Occurrences: 422

Product ID: B001AS1A4Q, Score: 4.21654501216545, Occurrences: 411

Product ID: B003D4F1QS, Score: 4.504926108374384, Occurrences: 406

Product ID: B000GAT6NG, Score: 4.802056555269923, Occurrences: 389

Product ID: B000ENUC3S, Score: 4.410852713178294,

# Summary - 25 most reviewed products with mean greater than 4 - 2008, 2009 & 2010

In [276]:
import pandas as pd

# Group by 'ProductId' and count occurrences, then sort by occurrence
product_occurrences = reviews[(reviews['Year'] == 2008) | (reviews['Year'] == 2009) | (reviews['Year'] == 2010)]['ProductId'].value_counts().sort_values(ascending=False)

# Sort the DataFrame based on product occurrences
sorted_df = reviews[reviews['ProductId'].isin(product_occurrences.index)].sort_values(by='ProductId', key=lambda x: product_occurrences[x], ascending=False)

#Calculate mean score for each product after sorting
mean_scores = sorted_df.groupby('ProductId', sort=False)['Score'].mean()

#Filter products with mean score equal to 1 and select the first 5 products
top_low_scored_products = mean_scores[mean_scores > 4].head(25)

client = AzureOpenAI(
    api_key="fec8056ea62842b2b3e1f787171f4f33",  
    api_version="2023-12-01-preview",
    azure_endpoint = "https://azureaistudioimagegeneration.openai.azure.com/"
    )
    
concatenated_response = ""
# Step 4: Print the ID, score, number of occurrences
for product_id, low_score in top_low_scored_products.items():
    occurrences = product_occurrences.get(product_id, 0)
    top_50_reviews = sorted_df[sorted_df['ProductId'] == product_id].nlargest(50, 'HelpfulnessNumerator')['CleantText'].tolist()
    concatenated_reviews = ' '.join(top_50_reviews)
    print(f"Product ID: {product_id}, Score: {low_score}, Occurrences: {occurrences}")

    response = client.chat.completions.create(
        model="capstone", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are going to answer only from the following reviews data" + concatenated_reviews},
            {"role": "user", "content": "Guess and name the product. Just rate from 1 to 5 and no text for the following aspects, Quality, Price, Brand, Packaging, Shipping, Value, Satisfaction, Health, Taste, Flavour. Then summarize what is positive shortly below"},
        ]
    )
    
    #print(response.choices[0].message.content)
    print()
    concatenated_response += response.choices[0].message.content
    
response = client.chat.completions.create(
        model="capstone", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are going to answer only from the following reviews data" + concatenated_response},
            {"role": "user", "content": "Give a general overview on the type of products, why rated positively and what can be improved. Give average rating of Quality, Price, Brand, Packaging, Shipping, Value, Satisfaction, Health, Taste, Flavour as a unordered list. Just list the product names at the last and no other content"},
        ]
    )

print(response.choices[0].message.content)

Product ID: B000NMJWZO, Score: 4.88909426987061, Occurrences: 342

Product ID: B000VK8AVK, Score: 4.307829181494662, Occurrences: 245

Product ID: B000UBD88A, Score: 4.343173431734317, Occurrences: 237

Product ID: B001EQ55RW, Score: 4.134831460674158, Occurrences: 222

Product ID: B001BCVY4W, Score: 4.262008733624454, Occurrences: 216

Product ID: B0029XLH4Y, Score: 4.256578947368421, Occurrences: 208

Product ID: B001OCKIP0, Score: 4.157407407407407, Occurrences: 208

Product ID: B000DZFMEQ, Score: 4.610169491525424, Occurrences: 187

Product ID: B000HDK0D2, Score: 4.528957528957529, Occurrences: 184

Product ID: B001EO5U3I, Score: 4.68169014084507, Occurrences: 178

Product ID: B001D0IZBM, Score: 4.197986577181208, Occurrences: 176

Product ID: B000FFIL92, Score: 4.242105263157895, Occurrences: 172

Product ID: B000HDK0DC, Score: 4.654275092936803, Occurrences: 170

Product ID: B000HDI5O8, Score: 4.031496062992126, Occurrences: 170

Product ID: B001AS1A4Q, Score: 4.21654501216545, O

# Summary - 25 most reviewed products with mean greater than 4 - 2011 & 2012

In [277]:
import pandas as pd

# Group by 'ProductId' and count occurrences, then sort by occurrence
product_occurrences = reviews[(reviews['Year'] == 2011) | (reviews['Year'] == 2012)]['ProductId'].value_counts().sort_values(ascending=False)

# Sort the DataFrame based on product occurrences
sorted_df = reviews[reviews['ProductId'].isin(product_occurrences.index)].sort_values(by='ProductId', key=lambda x: product_occurrences[x], ascending=False)

#Calculate mean score for each product after sorting
mean_scores = sorted_df.groupby('ProductId', sort=False)['Score'].mean()

#Filter products with mean score equal to 1 and select the first 5 products
top_low_scored_products = mean_scores[mean_scores > 4].head(25)

client = AzureOpenAI(
    api_key="fec8056ea62842b2b3e1f787171f4f33",  
    api_version="2023-12-01-preview",
    azure_endpoint = "https://azureaistudioimagegeneration.openai.azure.com/"
    )
    
concatenated_response = ""
# Step 4: Print the ID, score, number of occurrences
for product_id, low_score in top_low_scored_products.items():
    occurrences = product_occurrences.get(product_id, 0)
    top_50_reviews = sorted_df[sorted_df['ProductId'] == product_id].nlargest(50, 'HelpfulnessNumerator')['CleantText'].tolist()
    concatenated_reviews = ' '.join(top_50_reviews)
    print(f"Product ID: {product_id}, Score: {low_score}, Occurrences: {occurrences}")

    response = client.chat.completions.create(
        model="capstone", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are going to answer only from the following reviews data" + concatenated_reviews},
            {"role": "user", "content": "Guess and name the product. Just rate from 1 to 5 and no text for the following aspects, Quality, Price, Brand, Packaging, Shipping, Value, Satisfaction, Health, Taste, Flavour. Then summarize what is positive shortly below"},
        ]
    )
    
    #print(response.choices[0].message.content)
    print()
    concatenated_response += response.choices[0].message.content
    
response = client.chat.completions.create(
        model="capstone", # model = "deployment_name".
        messages=[
            {"role": "system", "content": "You are going to answer only from the following reviews data" + concatenated_response},
            {"role": "user", "content": "Give a general overview on the type of products, why rated positively and what can be improved. Give average rating of Quality, Price, Brand, Packaging, Shipping, Value, Satisfaction, Health, Taste, Flavour as a unordered list. Just list the product names at the last and no other content"},
        ]
    )

print(response.choices[0].message.content)

Product ID: B007JFMH8M, Score: 4.582236842105263, Occurrences: 912

Product ID: B003B3OOPA, Score: 4.739967897271268, Occurrences: 575

Product ID: B005ZBZLT4, Score: 4.364541832669323, Occurrences: 502

Product ID: B0026RQTGE, Score: 4.593650793650793, Occurrences: 472

Product ID: B001EO5Q64, Score: 4.746902654867257, Occurrences: 403

Product ID: B0051COPH6, Score: 4.469656992084433, Occurrences: 379

Product ID: B003D4F1QS, Score: 4.504926108374384, Occurrences: 376

Product ID: B000PDWBKO, Score: 4.3539094650205765, Occurrences: 368

Product ID: B006H34CUS, Score: 4.405970149253731, Occurrences: 335

Product ID: B000VK8AVK, Score: 4.307829181494662, Occurrences: 317

Product ID: B0007A0AQM, Score: 4.542654028436019, Occurrences: 306

Product ID: B004BKLHOS, Score: 4.274834437086093, Occurrences: 302

Product ID: B000UBD88A, Score: 4.343173431734317, Occurrences: 298

Product ID: B003XDH6M6, Score: 4.023489932885906, Occurrences: 297

Product ID: B004JGQ15E, Score: 4.20608108108108